In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import json
import os
# from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
pd.options.display.max_columns = None
import lightgbm as lgbm
from sklearn.model_selection import train_test_split


# from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, evaluate_model, finalize_model, predict_model, save_model, load_model

In [2]:
# with open('./.kaggle/kaggle.json') as file:
#     credentialsKaggle = json.load(file)

# os.environ['KAGGLE_USERNAME'] = credentialsKaggle['username']
# os.environ['KAGGLE_KEY'] = credentialsKaggle['key']

In [41]:
# # Descarga de la información
# api = KaggleApi()
# api.authenticate()

# # Download the competition files
# competition_name = 'playground-series-s4e6'
# download_path = 'data/'
# api.competition_download_files(competition_name, path=download_path)

# # Unzip the downloaded files
# for item in os.listdir(download_path):
#     if item.endswith('.zip'):
#         zip_ref = zipfile.ZipFile(os.path.join(download_path, item), 'r')
#         zip_ref.extractall(download_path)
#         zip_ref.close()
#         print(f"Unzipped {item}")

Unzipped playground-series-s4e6.zip


In [2]:
with open('dict_nombres.json') as file:
    dict_nombres = json.load(file)
dict_nombres

{'Marital status': 'est_civil',
 'Application mode': 'modo_solicitud',
 'Application order': 'orden_solicitud',
 'Course': 'curso',
 'Daytime/evening attendance': 'tip_asistencia',
 'Previous qualification': 'estudio_previo',
 'Previous qualification (grade)': 'nota_estudio_previo',
 'Nacionality': 'nacionalidad',
 "Mother's qualification": 'estudio_madre',
 "Father's qualification": 'estudio_padre',
 "Mother's occupation": 'profesion_madre',
 "Father's occupation": 'profesion_padre',
 'Admission grade': 'nota_admision',
 'Displaced': 'trasladado',
 'Educational special needs': 'necesidad_educativa_especial',
 'Debtor': 'deudor',
 'Tuition fees up to date': 'pension_al_dia',
 'Gender': 'genero',
 'Scholarship holder': 'becario',
 'Age at enrollment': 'edad_inscripcion',
 'International': 'internacional',
 'Curricular units 1st sem (credited)': 'num_curso_acreditado_primer_semestre',
 'Curricular units 1st sem (enrolled)': 'num_curso_matriculado_primer_semestre',
 'Curricular units 1st 

In [74]:
dfTemp_Train = pd.read_csv('./train.csv')
dfTemp_Train['flag_train'] = 1
dfTemp_Train

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,flag_train
0,0,1,1,1,9238,1,1,126.0,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.500000,0,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate,1
1,1,1,17,1,9238,1,1,125.0,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.600000,0,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout,1
2,2,1,17,2,9254,1,1,137.0,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout,1
3,3,1,1,3,9500,1,1,131.0,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.591250,0,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled,1
4,4,1,1,2,9500,1,1,132.0,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.933333,0,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,1,7,5,116.5,1,0,0,1,0,1,18,0,0,6,9,6,10.666667,0,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate,1
76514,76514,1,1,6,9254,1,1,125.0,1,1,38,4,9,131.6,1,0,0,1,0,0,19,0,0,6,22,4,13.000000,0,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate,1
76515,76515,5,17,1,9085,1,1,138.0,1,37,37,9,10,123.3,1,0,0,1,0,0,19,0,0,5,13,4,12.500000,2,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled,1
76516,76516,1,1,3,9070,1,1,136.0,1,38,37,5,9,124.8,1,0,0,1,0,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout,1


In [75]:
dfTemp_Test = pd.read_csv('./test.csv')
dfTemp_Test['flag_train'] = 0
dfTemp_Test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,flag_train
0,76518,1,1,1,9500,1,1,141.0,1,3,1,2,3,152.1,0,0,0,0,0,0,18,0,0,7,0,0,0.000000,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79,0
1,76519,1,1,1,9238,1,1,128.0,1,1,19,4,5,116.5,0,0,0,1,0,0,19,0,0,6,7,6,14.857143,0,0,6,6,6,13.500000,0,11.1,0.6,2.02,0
2,76520,1,1,1,9238,1,1,118.0,1,1,19,4,9,114.2,0,0,0,1,0,1,18,0,0,6,11,6,12.000000,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06,0
3,76521,1,44,1,9147,1,39,130.0,1,1,19,3,3,130.0,0,0,0,1,0,1,23,0,2,6,15,5,11.500000,0,3,8,14,5,11.000000,0,8.9,1.4,3.51,0
4,76522,1,39,1,9670,1,1,110.0,1,1,37,5,5,106.0,1,0,0,1,0,0,26,0,0,6,9,3,11.000000,0,0,6,9,4,10.666667,2,7.6,2.6,0.32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,37,7,10,124.7,1,0,0,1,0,0,19,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,0
51008,127526,2,39,1,9119,1,19,133.1,1,19,37,9,9,140.0,0,0,1,0,1,0,33,0,0,5,6,0,0.000000,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12,0
51009,127527,1,1,1,171,1,1,127.0,1,1,1,4,10,120.4,0,0,1,0,0,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,0
51010,127528,1,1,3,9773,1,1,132.0,1,19,19,5,5,126.3,1,0,1,0,0,0,18,0,0,6,8,5,12.600000,0,0,6,9,3,13.000000,0,7.6,2.6,0.32,0


In [80]:
dfTemp_concat = pd.concat([dfTemp_Train, dfTemp_Test], ignore_index=True)
dfTemp_concat

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,flag_train
0,0,1,1,1,9238,1,1,126.0,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.500000,0,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate,1
1,1,1,17,1,9238,1,1,125.0,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.600000,0,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout,1
2,2,1,17,2,9254,1,1,137.0,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout,1
3,3,1,1,3,9500,1,1,131.0,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.591250,0,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled,1
4,4,1,1,2,9500,1,1,132.0,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.933333,0,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127525,127525,1,1,2,171,1,1,128.0,1,38,37,7,10,124.7,1,0,0,1,0,0,19,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,NaN,0
127526,127526,2,39,1,9119,1,19,133.1,1,19,37,9,9,140.0,0,0,1,0,1,0,33,0,0,5,6,0,0.000000,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12,NaN,0
127527,127527,1,1,1,171,1,1,127.0,1,1,1,4,10,120.4,0,0,1,0,0,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,NaN,0
127528,127528,1,1,3,9773,1,1,132.0,1,19,19,5,5,126.3,1,0,1,0,0,0,18,0,0,6,8,5,12.600000,0,0,6,9,3,13.000000,0,7.6,2.6,0.32,NaN,0


In [81]:
dfTemp_concat.rename(columns = dict_nombres, inplace = True) 
dfTemp_concat

,id,est_civil,modo_solicitud,orden_solicitud,curso,tip_asistencia,estudio_previo,nota_estudio_previo,nacionalidad,estudio_madre,estudio_padre,profesion_madre,profesion_padre,nota_admision,trasladado,necesidad_educativa_especial,deudor,pension_al_dia,genero,becario,edad_inscripcion,internacional,num_curso_acreditado_primer_semestre,num_curso_matriculado_primer_semestre,num_curso_evaluado_primer_semestre,num_curso_aprobado_primer_semestre,num_curso_calificado_primer_semestre,num_curso_sin_evaluacion_primer_semestre,num_curso_acreditado_segundo_semestre,num_curso_matriculado_segundo_semestre,num_curso_evaluado_segundo_semestre,num_curso_aprobado_segundo_semestre,num_curso_calificado_segundo_semestre,num_curso_sin_evaluacion_segundo_semestre,ratio_desempleo,ratio_inflacion,pbi,target,flag_train
0,0,1,1,1,9238,1,1,126.0,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.500000,0,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate,1
1,1,1,17,1,9238,1,1,125.0,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.600000,0,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout,1
2,2,1,17,2,9254,1,1,137.0,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout,1
3,3,1,1,3,9500,1,1,131.0,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.591250,0,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled,1
4,4,1,1,2,9500,1,1,132.0,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.933333,0,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127525,127525,1,1,2,171,1,1,128.0,1,38,37,7,10,124.7,1,0,0,1,0,0,19,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,NaN,0
127526,127526,2,39,1,9119,1,19,133.1,1,19,37,9,9,140.0,0,0,1,0,1,0,33,0,0,5,6,0,0.000000,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12,NaN,0
127527,127527,1,1,1,171,1,1,127.0,1,1,1,4,10,120.4,0,0,1,0,0,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06,NaN,0
127528,127528,1,1,3,9773,1,1,132.0,1,19,19,5,5,126.3,1,0,1,0,0,0,18,0,0,6,8,5,12.600000,0,0,6,9,3,13.000000,0,7.6,2.6,0.32,NaN,0


In [82]:
col_numericas = [
    'nota_estudio_previo',
    'nota_admision',
    'num_curso_acreditado_primer_semestre',
    'num_curso_matriculado_primer_semestre',
    'num_curso_evaluado_primer_semestre',
    'num_curso_aprobado_primer_semestre', 
    'num_curso_calificado_primer_semestre',
    'num_curso_sin_evaluacion_primer_semestre',
    'num_curso_acreditado_segundo_semestre',
    'num_curso_matriculado_segundo_semestre',
    'num_curso_evaluado_segundo_semestre',
    'num_curso_aprobado_segundo_semestre', 
    'num_curso_calificado_segundo_semestre',
    'num_curso_sin_evaluacion_segundo_semestre',
    'ratio_desempleo',
    'ratio_inflacion',
    'pbi'
]

col_categoricas = [
    'est_civil',
    'modo_solicitud',
    'curso',
    'estudio_previo',
    'nacionalidad',
    'estudio_madre',
    'estudio_padre',
    'profesion_madre',
    'profesion_padre'
    ]

col_ordinales = ['orden_solicitud']

col_dummies = [
    'tip_asistencia',
    'trasladado', 
    'necesidad_educativa_especial', 
    'deudor',
    'pension_al_dia',
    'genero',
    'becario',
    'internacional'
    ]

col_target = 'target' 


In [83]:
for col in col_categoricas:
    dfDummies = pd.get_dummies(dfTemp_concat[col], prefix = col).astype(int)
    dfTemp_concat = pd.concat([dfTemp_concat, dfDummies], axis = 1)

dfTemp_concat

,id,est_civil,modo_solicitud,orden_solicitud,curso,tip_asistencia,estudio_previo,nota_estudio_previo,nacionalidad,estudio_madre,estudio_padre,profesion_madre,profesion_padre,nota_admision,trasladado,necesidad_educativa_especial,deudor,pension_al_dia,genero,becario,edad_inscripcion,internacional,num_curso_acreditado_primer_semestre,num_curso_matriculado_primer_semestre,num_curso_evaluado_primer_semestre,num_curso_aprobado_primer_semestre,num_curso_calificado_primer_semestre,num_curso_sin_evaluacion_primer_semestre,num_curso_acreditado_segundo_semestre,num_curso_matriculado_segundo_semestre,num_curso_evaluado_segundo_semestre,num_curso_aprobado_segundo_semestre,num_curso_calificado_segundo_semestre,num_curso_sin_evaluacion_segundo_semestre,ratio_desempleo,ratio_inflacion,pbi,target,flag_train,est_civil_1,est_civil_2,est_civil_3,est_civil_4,est_civil_5,est_civil_6,modo_solicitud_1,modo_solicitud_2,modo_solicitud_3,modo_solicitud_4,modo_solicitud_5,modo_solicitud_7,modo_solicitud_9,modo_solicitud_10,modo_solicitud_12,modo_solicitud_14,modo_solicitud_15,modo_solicitud_16,modo_solicitud_17,modo_solicitud_18,modo_solicitud_19,modo_solicitud_26,modo_solicitud_27,modo_solicitud_35,modo_solicitud_39,modo_solicitud_42,modo_solicitud_43,modo_solicitud_44,modo_solicitud_51,modo_solicitud_53,curso_33,curso_39,curso_171,curso_979,curso_2105,curso_4147,curso_7500,curso_8014,curso_9003,curso_9070,curso_9085,curso_9119,curso_9130,curso_9147,curso_9238,curso_9254,curso_9257,curso_9500,curso_9556,curso_9670,curso_9773,curso_9853,curso_9991,estudio_previo_1,estudio_previo_2,estudio_previo_3,estudio_previo_4,estudio_previo_5,estudio_previo_6,estudio_previo_9,estudio_previo_10,estudio_previo_11,estudio_previo_12,estudio_previo_14,estudio_previo_15,estudio_previo_16,estudio_previo_17,estudio_previo_19,estudio_previo_36,estudio_previo_37,estudio_previo_38,estudio_previo_39,estudio_previo_40,estudio_previo_42,estudio_previo_43,nacionalidad_1,nacionalidad_2,nacionalidad_6,nacionalidad_11,nacionalidad_14,nacionalidad_17,nacionalidad_21,nacionalidad_22,nacionalidad_24,nacionalidad_25,nacionalidad_26,nacionalidad_32,nacionalidad_41,nacionalidad_62,nacionalidad_100,nacionalidad_101,nacionalidad_103,nacionalidad_105,nacionalidad_109,estudio_madre_1,estudio_madre_2,estudio_madre_3,estudio_madre_4,estudio_madre_5,estudio_madre_6,estudio_madre_7,estudio_madre_8,estudio_madre_9,estudio_madre_10,estudio_madre_11,estudio_madre_12,estudio_madre_13,estudio_madre_14,estudio_madre_15,estudio_madre_18,estudio_madre_19,estudio_madre_22,estudio_madre_25,estudio_madre_26,estudio_madre_27,estudio_madre_28,estudio_madre_29,estudio_madre_30,estudio_madre_31,estudio_madre_33,estudio_madre_34,estudio_madre_35,estudio_madre_36,estudio_madre_37,estudio_madre_38,estudio_madre_39,estudio_madre_40,estudio_madre_41,estudio_madre_42,estudio_madre_43,estudio_madre_44,estudio_padre_1,estudio_padre_2,estudio_padre_3,estudio_padre_4,estudio_padre_5,estudio_padre_6,estudio_padre_7,estudio_padre_9,estudio_padre_10,estudio_padre_11,estudio_padre_12,estudio_padre_13,estudio_padre_14,estudio_padre_15,estudio_padre_16,estudio_padre_18,estudio_padre_19,estudio_padre_20,estudio_padre_21,estudio_padre_22,estudio_padre_23,estudio_padre_24,estudio_padre_25,estudio_padre_26,estudio_padre_27,estudio_padre_28,estudio_padre_29,estudio_padre_30,estudio_padre_31,estudio_padre_33,estudio_padre_34,estudio_padre_35,estudio_padre_36,estudio_padre_37,estudio_padre_38,estudio_padre_39,estudio_padre_40,estudio_padre_41,estudio_padre_42,estudio_padre_43,estudio_padre_44,profesion_madre_0,profesion_madre_1,profesion_madre_2,profesion_madre_3,profesion_madre_4,profesion_madre_5,profesion_madre_6,profesion_madre_7,profesion_madre_8,profesion_madre_9,profesion_madre_10,profesion_madre_11,profesion_madre_38,profesion_madre_90,profesion_madre_98,profesion_madre_99,profesion_madre_101,profesion_madre_103,profesion_madre_122,profesion_madre_123,profesion_madre_124,profesion_madre_125,profesion_madre_127,profesion_madre_

In [84]:
dfTemp_concat['est_civil_relacion'] = dfTemp_concat.apply(lambda row: 1 if row['est_civil'] in (2,5) else 0, axis = 1)
dfTemp_concat

,id,est_civil,modo_solicitud,orden_solicitud,curso,tip_asistencia,estudio_previo,nota_estudio_previo,nacionalidad,estudio_madre,estudio_padre,profesion_madre,profesion_padre,nota_admision,trasladado,necesidad_educativa_especial,deudor,pension_al_dia,genero,becario,edad_inscripcion,internacional,num_curso_acreditado_primer_semestre,num_curso_matriculado_primer_semestre,num_curso_evaluado_primer_semestre,num_curso_aprobado_primer_semestre,num_curso_calificado_primer_semestre,num_curso_sin_evaluacion_primer_semestre,num_curso_acreditado_segundo_semestre,num_curso_matriculado_segundo_semestre,num_curso_evaluado_segundo_semestre,num_curso_aprobado_segundo_semestre,num_curso_calificado_segundo_semestre,num_curso_sin_evaluacion_segundo_semestre,ratio_desempleo,ratio_inflacion,pbi,target,flag_train,est_civil_1,est_civil_2,est_civil_3,est_civil_4,est_civil_5,est_civil_6,modo_solicitud_1,modo_solicitud_2,modo_solicitud_3,modo_solicitud_4,modo_solicitud_5,modo_solicitud_7,modo_solicitud_9,modo_solicitud_10,modo_solicitud_12,modo_solicitud_14,modo_solicitud_15,modo_solicitud_16,modo_solicitud_17,modo_solicitud_18,modo_solicitud_19,modo_solicitud_26,modo_solicitud_27,modo_solicitud_35,modo_solicitud_39,modo_solicitud_42,modo_solicitud_43,modo_solicitud_44,modo_solicitud_51,modo_solicitud_53,curso_33,curso_39,curso_171,curso_979,curso_2105,curso_4147,curso_7500,curso_8014,curso_9003,curso_9070,curso_9085,curso_9119,curso_9130,curso_9147,curso_9238,curso_9254,curso_9257,curso_9500,curso_9556,curso_9670,curso_9773,curso_9853,curso_9991,estudio_previo_1,estudio_previo_2,estudio_previo_3,estudio_previo_4,estudio_previo_5,estudio_previo_6,estudio_previo_9,estudio_previo_10,estudio_previo_11,estudio_previo_12,estudio_previo_14,estudio_previo_15,estudio_previo_16,estudio_previo_17,estudio_previo_19,estudio_previo_36,estudio_previo_37,estudio_previo_38,estudio_previo_39,estudio_previo_40,estudio_previo_42,estudio_previo_43,nacionalidad_1,nacionalidad_2,nacionalidad_6,nacionalidad_11,nacionalidad_14,nacionalidad_17,nacionalidad_21,nacionalidad_22,nacionalidad_24,nacionalidad_25,nacionalidad_26,nacionalidad_32,nacionalidad_41,nacionalidad_62,nacionalidad_100,nacionalidad_101,nacionalidad_103,nacionalidad_105,nacionalidad_109,estudio_madre_1,estudio_madre_2,estudio_madre_3,estudio_madre_4,estudio_madre_5,estudio_madre_6,estudio_madre_7,estudio_madre_8,estudio_madre_9,estudio_madre_10,estudio_madre_11,estudio_madre_12,estudio_madre_13,estudio_madre_14,estudio_madre_15,estudio_madre_18,estudio_madre_19,estudio_madre_22,estudio_madre_25,estudio_madre_26,estudio_madre_27,estudio_madre_28,estudio_madre_29,estudio_madre_30,estudio_madre_31,estudio_madre_33,estudio_madre_34,estudio_madre_35,estudio_madre_36,estudio_madre_37,estudio_madre_38,estudio_madre_39,estudio_madre_40,estudio_madre_41,estudio_madre_42,estudio_madre_43,estudio_madre_44,estudio_padre_1,estudio_padre_2,estudio_padre_3,estudio_padre_4,estudio_padre_5,estudio_padre_6,estudio_padre_7,estudio_padre_9,estudio_padre_10,estudio_padre_11,estudio_padre_12,estudio_padre_13,estudio_padre_14,estudio_padre_15,estudio_padre_16,estudio_padre_18,estudio_padre_19,estudio_padre_20,estudio_padre_21,estudio_padre_22,estudio_padre_23,estudio_padre_24,estudio_padre_25,estudio_padre_26,estudio_padre_27,estudio_padre_28,estudio_padre_29,estudio_padre_30,estudio_padre_31,estudio_padre_33,estudio_padre_34,estudio_padre_35,estudio_padre_36,estudio_padre_37,estudio_padre_38,estudio_padre_39,estudio_padre_40,estudio_padre_41,estudio_padre_42,estudio_padre_43,estudio_padre_44,profesion_madre_0,profesion_madre_1,profesion_madre_2,profesion_madre_3,profesion_madre_4,profesion_madre_5,profesion_madre_6,profesion_madre_7,profesion_madre_8,profesion_madre_9,profesion_madre_10,profesion_madre_11,profesion_madre_38,profesion_madre_90,profesion_madre_98,profesion_madre_99,profesion_madre_101,profesion_madre_103,profesion_madre_122,profesion_madre_123,profesion_madre_124,profesion_madre_125,profesion_madre_127,profesion_madre_

In [85]:
dfTrain = dfTemp_concat[dfTemp_concat['flag_train'] == 1].copy().drop(columns = 'flag_train')
dfTest = dfTemp_concat[dfTemp_concat['flag_train'] == 0].copy().drop(columns = 'flag_train')

In [88]:
X = dfTrain.drop(columns = ['id','target'])
y = dfTrain['target']

In [89]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [90]:
clf = lgbm.LGBMClassifier(learning_rate=0.09,max_depth=5,n_estimators = 100,n_jobs = -1, objective = 'multiclass', 
                          eval_metric= 'acurracy', verbose= 200, random_state=123)
clf.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.951676
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.604851
[LightGBM] [Debug] init for col-wise cost 0.028503 seconds, init for row-wise cost 0.019172 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020423 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1606
[LightGBM] [Info] Number of data points in the train set: 53562, number of used features: 156
[LightGBM] [Info] Start training from score -1.101810
[LightGBM] [Info] Start training from score -1.638169
[LightGBM] [Info] Start training from score -0.747824
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and de

LGBMClassifier(eval_metric='acurracy', learning_rate=0.09, max_depth=5,
               n_jobs=-1, objective='multiclass', random_state=123,
               verbose=200)

In [113]:
from sklearn.model_selection import RandomizedSearchCV as RCV

from sklearn.model_selection import RepeatedStratifiedKFold

In [114]:
param_dist_lgbm = {
    'n_estimators': [100, 200,300,400,500],
    'max_depth': [3, 5,10, 15],
    'min_samples_split': [2, 5, 10,15,20], 
    'learning_rate':['0.01','0.05','0.1']
}
n_iter = 50
n_jobs = -1
verbose = 1
random_state = 123
scoring = 'accuracy'
param_distributions = param_dist_lgbm

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=123)

random_search = RCV(estimator = clf, n_iter = n_iter, n_jobs = n_jobs, cv = cv, verbose = verbose, 
    random_state = random_state, scoring = scoring, param_distributions = param_distributions)

In [115]:
random_search.fit(X_train, y_train)

Fitting 30 folds for each of 50 candidates, totalling 1500 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.951676
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.604851
[LightGBM] [Debug] init for col-wise cost 0.003046 seconds, init for row-wise cost 0.025064 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004809 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1606
[LightGBM] [Info] Number of data points in the train set: 53562, number of used features: 156
[LightGBM] [Info] Start training from score -1.101810
[Ligh

RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=123),
                   estimator=LGBMClassifier(eval_metric='acurracy',
                                            learning_rate=0.09, max_depth=5,
                                            n_jobs=-1, objective='multiclass',
                                            random_state=123, verbose=200),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'learning_rate': ['0.01', '0.05',
                                                          '0.1'],
                                        'max_depth': [3, 5, 10, 15],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=123, scoring='accuracy', verbose=1)

In [116]:
print(random_search.best_score_)
print(random_search.best_params_)
print(random_search.best_estimator_)

0.8334328055735125
{'n_estimators': 300, 'min_samples_split': 10, 'max_depth': 5, 'learning_rate': '0.05'}
LGBMClassifier(eval_metric='acurracy', learning_rate='0.05', max_depth=5,
               min_samples_split=10, n_estimators=300, n_jobs=-1,
               objective='multiclass', random_state=123, verbose=200)


In [117]:
model_final = random_search.best_estimator_
model_final

LGBMClassifier(eval_metric='acurracy', learning_rate='0.05', max_depth=5,
               min_samples_split=10, n_estimators=300, n_jobs=-1,
               objective='multiclass', random_state=123, verbose=200)

In [118]:
y_pred=model_final.predict(X_test)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split


In [119]:
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.8283


In [120]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(model_final.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(model_final.score(X_test, y_test)))

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split
Training set score: 0.8547
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split
Test set score: 0.8283


In [121]:
model_final.n_features_

312

In [122]:
dfTest

,id,est_civil,modo_solicitud,orden_solicitud,curso,tip_asistencia,estudio_previo,nota_estudio_previo,nacionalidad,estudio_madre,estudio_padre,profesion_madre,profesion_padre,nota_admision,trasladado,necesidad_educativa_especial,deudor,pension_al_dia,genero,becario,edad_inscripcion,internacional,num_curso_acreditado_primer_semestre,num_curso_matriculado_primer_semestre,num_curso_evaluado_primer_semestre,num_curso_aprobado_primer_semestre,num_curso_calificado_primer_semestre,num_curso_sin_evaluacion_primer_semestre,num_curso_acreditado_segundo_semestre,num_curso_matriculado_segundo_semestre,num_curso_evaluado_segundo_semestre,num_curso_aprobado_segundo_semestre,num_curso_calificado_segundo_semestre,num_curso_sin_evaluacion_segundo_semestre,ratio_desempleo,ratio_inflacion,pbi,target,est_civil_1,est_civil_2,est_civil_3,est_civil_4,est_civil_5,est_civil_6,modo_solicitud_1,modo_solicitud_2,modo_solicitud_3,modo_solicitud_4,modo_solicitud_5,modo_solicitud_7,modo_solicitud_9,modo_solicitud_10,modo_solicitud_12,modo_solicitud_14,modo_solicitud_15,modo_solicitud_16,modo_solicitud_17,modo_solicitud_18,modo_solicitud_19,modo_solicitud_26,modo_solicitud_27,modo_solicitud_35,modo_solicitud_39,modo_solicitud_42,modo_solicitud_43,modo_solicitud_44,modo_solicitud_51,modo_solicitud_53,curso_33,curso_39,curso_171,curso_979,curso_2105,curso_4147,curso_7500,curso_8014,curso_9003,curso_9070,curso_9085,curso_9119,curso_9130,curso_9147,curso_9238,curso_9254,curso_9257,curso_9500,curso_9556,curso_9670,curso_9773,curso_9853,curso_9991,estudio_previo_1,estudio_previo_2,estudio_previo_3,estudio_previo_4,estudio_previo_5,estudio_previo_6,estudio_previo_9,estudio_previo_10,estudio_previo_11,estudio_previo_12,estudio_previo_14,estudio_previo_15,estudio_previo_16,estudio_previo_17,estudio_previo_19,estudio_previo_36,estudio_previo_37,estudio_previo_38,estudio_previo_39,estudio_previo_40,estudio_previo_42,estudio_previo_43,nacionalidad_1,nacionalidad_2,nacionalidad_6,nacionalidad_11,nacionalidad_14,nacionalidad_17,nacionalidad_21,nacionalidad_22,nacionalidad_24,nacionalidad_25,nacionalidad_26,nacionalidad_32,nacionalidad_41,nacionalidad_62,nacionalidad_100,nacionalidad_101,nacionalidad_103,nacionalidad_105,nacionalidad_109,estudio_madre_1,estudio_madre_2,estudio_madre_3,estudio_madre_4,estudio_madre_5,estudio_madre_6,estudio_madre_7,estudio_madre_8,estudio_madre_9,estudio_madre_10,estudio_madre_11,estudio_madre_12,estudio_madre_13,estudio_madre_14,estudio_madre_15,estudio_madre_18,estudio_madre_19,estudio_madre_22,estudio_madre_25,estudio_madre_26,estudio_madre_27,estudio_madre_28,estudio_madre_29,estudio_madre_30,estudio_madre_31,estudio_madre_33,estudio_madre_34,estudio_madre_35,estudio_madre_36,estudio_madre_37,estudio_madre_38,estudio_madre_39,estudio_madre_40,estudio_madre_41,estudio_madre_42,estudio_madre_43,estudio_madre_44,estudio_padre_1,estudio_padre_2,estudio_padre_3,estudio_padre_4,estudio_padre_5,estudio_padre_6,estudio_padre_7,estudio_padre_9,estudio_padre_10,estudio_padre_11,estudio_padre_12,estudio_padre_13,estudio_padre_14,estudio_padre_15,estudio_padre_16,estudio_padre_18,estudio_padre_19,estudio_padre_20,estudio_padre_21,estudio_padre_22,estudio_padre_23,estudio_padre_24,estudio_padre_25,estudio_padre_26,estudio_padre_27,estudio_padre_28,estudio_padre_29,estudio_padre_30,estudio_padre_31,estudio_padre_33,estudio_padre_34,estudio_padre_35,estudio_padre_36,estudio_padre_37,estudio_padre_38,estudio_padre_39,estudio_padre_40,estudio_padre_41,estudio_padre_42,estudio_padre_43,estudio_padre_44,profesion_madre_0,profesion_madre_1,profesion_madre_2,profesion_madre_3,profesion_madre_4,profesion_madre_5,profesion_madre_6,profesion_madre_7,profesion_madre_8,profesion_madre_9,profesion_madre_10,profesion_madre_11,profesion_madre_38,profesion_madre_90,profesion_madre_98,profesion_madre_99,profesion_madre_101,profesion_madre_103,profesion_madre_122,profesion_madre_123,profesion_madre_124,profesion_madre_125,profesion_madre_127,profesion_madre_131,profesi

In [123]:
dfPredict = model_final.predict(dfTest.drop(columns = ['id','target']))

result = pd.DataFrame({'id' : dfTest['id'], 'Target': dfPredict})

result.to_csv( 'submission_2.csv', index = False)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: min_samples_split
